# 유투브 랭킹 데이터 수집해서 시각화 하기

In [36]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [37]:
options = webdriver.ChromeOptions()
#options.add_argument("--headless")      # 화면없이 실행
#options.add_argument("--no-sandbox")
#options.add_argument("--single-process")
#options.add_argument("--disable-dev-shm-usage")
#driver = webdriver.Chrome("chromedriver.exe", options=options)
driver = webdriver.Chrome("chromedriver.exe")

In [38]:

url = "https://www.youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1"
driver.get(url)

In [41]:
trs = driver.find_elements_by_css_selector(".aos-init")
len(trs)

100

In [42]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [43]:
channel_list = soup.select(".aos-init")
len(channel_list)

100

In [44]:
channel = channel_list[0]
category = channel.select_one("p.category").get_text().strip()
category

'[음악/댄스/가수]'

In [45]:
name = channel.select_one(".subject a").text.strip()
name

'BLACKPINK'

In [46]:
subscriber = channel.select_one(".subscriber_cnt").text
view = channel.select_one(".view_cnt").text
video = channel.select_one(".video_cnt").text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [47]:
channels = []
for channel in channel_list:
    category = channel.select_one("p.category").get_text().strip(" \n[]")
    name = channel.select_one(".subject a").text.strip()
    subscriber = channel.select_one(".subscriber_cnt").text
    view = channel.select_one(".view_cnt").text
    video = channel.select_one(".video_cnt").text[:-1]
    channels.append([category, name, subscriber, view, video])

In [48]:
df = pd.DataFrame(channels, columns = ["카테고리","채널명","구독자수","조회수","비디오수"])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [49]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,애완/반려동물,크림히어로즈,381만,13억7401만,939


- 만과 억을 숫자로 바꿔주는 함수

In [50]:
def convert_unit(s):
    #s = "".join(s.split("억"))
    s = s.replace("억","").replace("개","").replace(",","")
    s = s.replace("만","0000")
    return f"{int(s):d}"

In [51]:
convert_unit("10억6557만")

'1065570000'

- 두번째 페이지 - XPath로 찾기

In [52]:
# //*[@id="list-skin"]/nav/span/a[1]
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [53]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [58]:
results = []
for channel in channel_list:
    category = channel.select_one("p.category").get_text().strip(" \n[]")
    name = channel.select_one(".subject a").text.strip()
    subscriber = convert_unit(channel.select_one(".subscriber_cnt").text)
    view = channel.select_one(".view_cnt").text
    video = channel.select_one(".video_cnt").text[:-1]
    results.append([category, name, subscriber, view, video])

In [59]:
df = pd.DataFrame(results, columns = ["카테고리","채널명","구독자수","조회수","비디오수"])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,64000000,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,60300000,187억1305만,654
2,음악/댄스/가수,BANGTANTV,56400000,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,28500000,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,110억5288만,223


## page 1에서 10 까지 크롤링 하기

In [70]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [71]:
df = pd.DataFrame(results, columns = ["카테고리","채널명","구독자수","조회수","비디오수"])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,64000000,19003810000,371
1,음악/댄스/가수,HYBE LABELS,60300000,18713050000,654
2,음악/댄스/가수,BANGTANTV,56400000,12194960000,1579
3,음악/댄스/가수,SMTOWN,28500000,21870920000,3729
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223


In [62]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,게임,미소,510000,2억1077만,"3,084"
96,미분류,밉지않은 관종언니,510000,6774만,170
97,미분류,복지마블TV [Welfare Marble],510000,4996만,240
98,음식/요리/레시피,Muggo,510000,1억8372만,"1,388"
99,게임,임선비,510000,2억3429만,"1,134"


In [72]:
df.to_csv("유투브_순위.csv", index=None)

In [73]:
driver.close()
#browser.close()